In [1]:
#Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error



In [10]:
#Load Data
df_train = pd.read_csv('data/train_data_w_macro.csv')
df_test = pd.read_csv('data/test_data_w_macro.csv')

#Prepare Features and Target
X_train = df_train.drop(['resale_price','date'], axis=1)
y_train = df_train['resale_price']

#Prepare Features and Target
X_test = df_test.drop(['resale_price','date'], axis=1)
y_test = df_test['resale_price']

X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 157062 entries, 0 to 157061
Data columns (total 16 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   storey_range                   157062 non-null  float64
 1   floor_area_sqm                 157062 non-null  float64
 2   lease_months_left              157062 non-null  int64  
 3   gdp in chained (2015) dollars  157062 non-null  float64
 4   assets                         157062 non-null  float64
 5   liabilities                    157062 non-null  float64
 6   mas core inflation measure     157062 non-null  float64
 7   m_from2017                     157062 non-null  float64
 8   town_tampines                  157062 non-null  int64  
 9   town_punggol                   157062 non-null  int64  
 10  town_sembawang                 157062 non-null  int64  
 11  town_choa chu kang             157062 non-null  int64  
 12  town_sengkang                 

In [16]:
from xgboost import XGBRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV
import pandas as pd
from sklearn.metrics import mean_absolute_error

In [17]:
# Define the parameter distribution for XGBoost
xgb_param_dist = {
    'n_estimators': [50, 100, 200, 300],
    'max_depth': [3, 5, 10, 15],
    'learning_rate': [0.01, 0.1, 0.2, 0.3],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'gamma': [0, 0.1, 0.2, 0.3],
    'min_child_weight': [1, 3, 5]
}

# Initialize RandomizedSearchCV for XGBoost
xgb_search = RandomizedSearchCV(
    estimator=XGBRegressor(random_state=42),
    param_distributions=xgb_param_dist,
    n_iter=50,
    scoring='neg_mean_absolute_error',
    cv=5,
    n_jobs=-1,
    verbose=2,
    random_state=42
)

# Fit the random search on the training data
xgb_search.fit(X_train, y_train)

# Get the best parameters and the best model
best_xgb_params = xgb_search.best_params_
best_xgb_model = xgb_search.best_estimator_

print("Best XGBoost Parameters:", best_xgb_params)

Fitting 5 folds for each of 50 candidates, totalling 250 fits
[CV] END colsample_bytree=1.0, gamma=0.1, learning_rate=0.1, max_depth=5, min_child_weight=5, n_estimators=50, subsample=1.0; total time=   0.5s
[CV] END colsample_bytree=1.0, gamma=0.1, learning_rate=0.1, max_depth=5, min_child_weight=5, n_estimators=50, subsample=1.0; total time=   0.5s
[CV] END colsample_bytree=1.0, gamma=0.1, learning_rate=0.1, max_depth=5, min_child_weight=5, n_estimators=50, subsample=1.0; total time=   0.5s
[CV] END colsample_bytree=1.0, gamma=0.1, learning_rate=0.1, max_depth=5, min_child_weight=5, n_estimators=50, subsample=1.0; total time=   0.5s
[CV] END colsample_bytree=1.0, gamma=0.1, learning_rate=0.1, max_depth=5, min_child_weight=5, n_estimators=50, subsample=1.0; total time=   0.6s
[CV] END colsample_bytree=1.0, gamma=0.1, learning_rate=0.01, max_depth=5, min_child_weight=1, n_estimators=200, subsample=0.6; total time=   1.7s
[CV] END colsample_bytree=1.0, gamma=0.1, learning_rate=0.01, max_

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[CV] END colsample_bytree=0.8, gamma=0, learning_rate=0.01, max_depth=10, min_child_weight=3, n_estimators=100, subsample=0.6; total time=   1.8s
[CV] END colsample_bytree=0.8, gamma=0, learning_rate=0.01, max_depth=10, min_child_weight=3, n_estimators=100, subsample=0.6; total time=   1.9s
[CV] END colsample_bytree=0.8, gamma=0, learning_rate=0.01, max_depth=10, min_child_weight=3, n_estimators=100, subsample=0.6; total time=   1.8s
[CV] END colsample_bytree=0.8, gamma=0, learning_rate=0.01, max_depth=10, min_child_weight=3, n_estimators=100, subsample=0.6; total time=   1.9s
[CV] END colsample_bytree=0.8, gamma=0, learning_rate=0.01, max_depth=10, min_child_weight=3, n_estimators=100, subsample=0.6; total time=   1.9s
[CV] END colsample_bytree=0.6, gamma=0, learning_rate=0.01, max_depth=15, min_child_weight=3, n_estimators=300, subsample=0.8; total time=  14.4s
[CV] END colsample_bytree=0.6, gamma=0, learning_rate=0.01, max_depth=15, min_child_weight=3, n_estimators=300, subsample=0.

In [19]:
# Initialize the XGBoost model with the best parameters
xgb_best_model = XGBRegressor(
    subsample=1.0,
    n_estimators=200,
    min_child_weight=5,
    max_depth=15,
    learning_rate=0.1,
    gamma=0.1,
    colsample_bytree=0.6,
    random_state=42
)

# Train the model on the training data
xgb_best_model.fit(X_train, y_train)

# Predict on the test data
y_pred_xgb_best = xgb_best_model.predict(X_test)

# ➡️ Save the trained model
xgb_best_model.save_model('model_trees/best_xgb_model_for_ensemble.json')

# Evaluate the model
print('\nXGBoost (Best Parameters) Performance:')
print(f'MSE: {mean_squared_error(y_test, y_pred_xgb_best):.2f}')
print(f'MAE: {mean_absolute_error(y_test, y_pred_xgb_best):.2f}')
print(f'R²: {r2_score(y_test, y_pred_xgb_best):.2f}')


XGBoost (Best Parameters) Performance:
MSE: 5074997439.76
MAE: 46833.09
R²: 0.84
